<a href="https://colab.research.google.com/github/deeplearningexplore/siamese/blob/main/question_pairs_detection2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install -q kaggle
from google.colab import files
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download -c quora-question-pairs

Saving kaggle.json to kaggle.json
 24% 5.00M/21.2M [00:00<00:02, 7.38MB/s]
100% 21.2M/21.2M [00:00<00:00, 27.2MB/s]
 92% 105M/114M [00:01<00:00, 50.0MB/s] 
100% 114M/114M [00:01<00:00, 60.7MB/s]
100% 4.95M/4.95M [00:00<00:00, 39.8MB/s]

test.csv.zip: Skipping, found more recently modified local copy (use --force to force download)


In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import zipfile
import pandas as pd
import tensorflow.keras.backend as K

%matplotlib inline

In [3]:
zip = zipfile.ZipFile("test.csv.zip", "r")
zip.extractall("./data")


In [4]:
zip = zipfile.ZipFile("train.csv.zip", "r")
zip.extractall("./data")

zip.close()

In [5]:
!rm "train.csv.zip"
!rm "test.csv.zip"

In [6]:
train_data = pd.read_csv("./data/train.csv")
test_data = pd.read_csv("./data/test.csv")
print(train_data.head())
print(test_data.head())

   id  qid1  ...                                          question2 is_duplicate
0   0     1  ...  What is the step by step guide to invest in sh...            0
1   1     3  ...  What would happen if the Indian government sto...            0
2   2     5  ...  How can Internet speed be increased by hacking...            0
3   3     7  ...  Find the remainder when [math]23^{24}[/math] i...            0
4   4     9  ...            Which fish would survive in salt water?            0

[5 rows x 6 columns]
   test_id  ...                                          question2
0        0  ...  Why did Microsoft choose core m3 and not core ...
1        1  ...        How much cost does hair transplant require?
2        2  ...                      What you send money to China?
3        3  ...                                  What foods fibre?
4        4  ...                     How their can I start reading?

[5 rows x 3 columns]


In [7]:
train_data.columns

Index(['id', 'qid1', 'qid2', 'question1', 'question2', 'is_duplicate'], dtype='object')

In [8]:
train_data = train_data[train_data["is_duplicate"]==1]
train_data.head()

,id,qid1,qid2,question1,question2,is_duplicate
5,5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1
7,7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1
11,11,23,24,How do I read and find my YouTube comments?,How can I see all my Youtube comments?,1
12,12,25,26,What can make Physics easy to learn?,How can you make physics easy to learn?,1
13,13,27,28,What was your first sexual experience like?,What was your first sexual experience?,1


In [9]:
train_d1 = train_data['question1'].values
train_d2 = train_data['question2'].values

In [10]:
train_d1.shape

(149263,)

In [11]:
train_d = np.concatenate([train_d1,train_d2])

In [12]:
train_d.shape

(298526,)

In [13]:
oov_token = "<ukw>"
pad = "post"


In [14]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(oov_token=oov_token)
tokenizer.fit_on_texts(train_d)
word_index = tokenizer.word_index


In [15]:
len(word_index)

31468

In [16]:
train_data1 = tokenizer.texts_to_sequences(train_d1)

In [17]:
train_data2 = tokenizer.texts_to_sequences(train_d2)

In [18]:
word_index["ambulance"]

19614

In [124]:
def data_generator(q1,q2,batch_size):
  indexes = list(range(len(q1)))
  indexes = tf.random.shuffle(indexes)
  length = len(q1)
  input1 = []
  input2 = []
  index = 0

  while True:
    if index==length:
      index = 0
      indexes = tf.random.shuffle(indexes)
    
    input1.append(q1[indexes[index]])
    input2.append(q2[indexes[index]])

    index+=1

    if len(input1)==batch_size:
      max_length = np.max((np.max([len(x) for x in input1]),np.max([len(x) for x in input2])))
      max_length = 2**int(np.ceil(np.log2(max_length)))

      input1 = tf.keras.preprocessing.sequence.pad_sequences(input1,maxlen=max_length,padding=pad,truncating=pad)
      input2 = tf.keras.preprocessing.sequence.pad_sequences(input2,maxlen=max_length,padding=pad,truncating=pad)

      yield [np.array(input1),np.array(input2)],np.zeros((2,batch_size, np.array(input1).shape[1]))

      input1 = []
      input2 = []



In [125]:
a,b=next(data_generator(train_data1,train_data2,5))

In [126]:
print(a[0].shape)
p = np.array(a)
print(p)
print(b)

(5, 16)
[[[    3    11     2    99    90     7   107   697     0     0     0
       0     0     0     0     0]
  [    3     4  2082   360    12    44     0     0     0     0     0
       0     0     0     0     0]
  [    3     8    15    65     4   177     9    44     0     0     0
       0     0     0     0     0]
  [    8    50    27     2  9531    36  2739   160    12     2   686
      28    50 10831   328  3401]
  [    5     6    13   359    73   731     0     0     0     0     0
       0     0     0     0     0]]

 [[    5     8     6   856   697     0     0     0     0     0     0
       0     0     0     0     0]
  [    3     4     2  1954   360    12    44     0     0     0     0
       0     0     0     0     0]
  [    3     4    48   177     7    10   161     9   226    44     0
       0     0     0     0     0]
  [    8    50    27     2  9531   160    12     2   686    28    50
   10831   328  3401     0     0]
  [    5     8     6   359    73    53  1788     0     0     0 

In [127]:
!wget "https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip"

--2020-10-15 04:05:23--  https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 104.22.75.142, 172.67.9.4, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 681808098 (650M) [application/zip]
Saving to: ‘wiki-news-300d-1M.vec.zip’

ki-news-300d-1M.vec   4%[                    ]  31.83M  9.35MB/s    eta 79s    ^C


In [128]:
zip = zipfile.ZipFile("wiki-news-300d-1M.vec.zip", "r")
zip.extractall("./data")
zip.close()

BadZipFile: ignored

In [38]:
!rm "wiki-news-300d-1M.vec.zip"

In [39]:
import io

def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = np.asarray(tokens[1:], dtype='float32')
    return data

embedding_vectors =  load_vectors("data/wiki-news-300d-1M.vec")

In [40]:
len(embedding_vectors)

999994

In [41]:
vocab_size =  len(tokenizer.word_index) + 1
dimensions = 300
batch_size = 128

In [42]:
embedding_matrix = np.zeros((vocab_size, dimensions))
for word,index in word_index.items():
  val = embedding_vectors.get(word, None)
  if val is not None:
    embedding_matrix[index] = val

In [43]:
del embedding_vectors

In [ ]:
# tf.compat.v1.enable_eager_execution()

In [129]:
embedding_layer = tf.keras.layers.Embedding(vocab_size,
                                            dimensions,
                                            weights=[embedding_matrix],
                                            trainable = False
                                            )

model = tf.keras.Sequential()
model.add(embedding_layer)
model.add(tf.keras.layers.LSTM(10))

input1 = tf.keras.Input(shape=(None,))
input2 = tf.keras.Input(shape=(None,))

vectors1 = model(input1)
vectors2 = model(input2)

output = tf.keras.layers.concatenate([vectors1, vectors2], axis=0)

saimese = tf.keras.Model(inputs = [input1, input2], outputs = output)

saimese.summary()


Model: "functional_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_10 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
sequential_4 (Sequential)       (None, 10)           9453140     input_9[0][0]                    
                                                                 input_10[0][0]                   
__________________________________________________________________________________________________
concatenate_4 (Concatenate)     (None, 10)           0           sequential_4[0][0]    

In [130]:
x = np.array(saimese([a[0],a[1]]))
x.shape

(10, 10)

In [131]:
def cosine_similarity(a,b):
  return np.dot(a,b)/(np.linalg.norm(a) * np.linalg.norm(b))

In [143]:
similarity_matrix1 = np.empty((0,batch_size),float)
for i in range(len(a[0])):
 row1 = []
 for j in range(len(a[1])):
  row1.append(cosine_similarity(a[0][i],a[1][j]))
  row1 = np.asarray(row1)
similarity_matrix1 = np.append(similarity_matrix1,row,axis=0)

AttributeError: ignored

In [136]:
def TripletLoss(batch_size, margin = 0.25):
  def lossfn(actual, vectors):
    anchor1 = vectors[:batch_size,:]
    anchor2 = vectors[batch_size:,:]

    '''anchor1 = anchor1 / K.expand_dims(K.sum(K.square(anchor1), axis=1), axis=-1)
    anchor2 = anchor2 / K.expand_dims(K.sum(K.square(anchor2), axis=1), axis =-1)
    similarity_matrix = K.dot(anchor1, K.transpose(anchor2))'''

    '''similarity_matrix = cosine_similarity(anchor1,anchor2)'''

    similarity_matrix = np.empty((0,batch_size),float)
    for i in range(batch_size):
      row = []
      for j in range(batch_size):
        row.append(cosine_similarity(anchor1[i],anchor2[j]))
      row = np.asarray(row)
      similarity_matrix = np.append(similarity_matrix,row,axis=0)

    #similarity_matrix = np.array(similarity_matrix)

    positive = K.sum(similarity_matrix * K.eye(batch_size), axis=1)
    temp1 = similarity_matrix - K.eye(batch_size) * 2.0
    temp2 = similarity_matrix * ( 1 - K.eye(batch_size))

    mean_negative = K.sum(temp2) / (batch_size-1)
    max_negative = K.max(temp1, axis=1)

    loss1 = K.maximum(0.0, mean_negative - positive + margin )
    loss2 = K.maximum(0.0, max_negative - positive + margin )

    loss = K.mean(loss1 + loss2)

    return loss
  return lossfn

In [137]:
saimese.compile(optimizer="adam", loss = TripletLoss(batch_size = batch_size))
tf.config.experimental_run_functions_eagerly(True)

In [138]:
saimese.fit_generator(data_generator(train_data1,train_data2,batch_size), epochs=10, steps_per_epoch=100)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/data/ops/dataset_ops.py:3350: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "


Epoch 1/10


ValueError: ignored

In [ ]:
def predict(a, b, margin=0):
  vectors = saimese([a,b])
  similarity = cosine_similarity(vectors[0], vectors[1])
  for i in similarity:
    if i > 0:
      print("similar")
    else: 
      print("Not Similar")
